# Course Project
## Bahaa Joudieh, Carly Craffen, Jordan Richardson & Vivian Wu

In [ ]:
# Import the necessary Python modules for data analysis and visualization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import missingno as msno
import pingouin as pg
pd.options.display.precision = 2

In [ ]:
# Read the required CSV files into their own dataframes
df_fuel_level_1 = pd.read_csv('https://www.dropbox.com/scl/fi/790m9gez90tjjsmiw27tn/Fuel_Level_Part_1.csv?rlkey=rsb3q82tsfqc7y6ds1qfy12h1&raw=1')
df_fuel_level_2 = pd.read_csv('https://www.dropbox.com/scl/fi/fjsdt4madtvref5xj18g0/Fuel_Level_Part_2.csv?rlkey=brch0h5n4cuv6qlpm9drbzxnn&raw=1')
df_invoices = pd.read_csv('https://www.dropbox.com/scl/fi/4q9jh3wmobkzr6ygbirx0/Invoices.csv?rlkey=tki3bfeeifstgxnjwptiwxsnb&raw=1')
df_locations = pd.read_csv('https://www.dropbox.com/scl/fi/cmiedj0s7dd7srn5ffpo6/Locations.csv?rlkey=f6lbol9mwc8t1l8f2f2itr7xf&raw=1')
df_tanks = pd.read_csv('https://www.dropbox.com/scl/fi/4x6hvawziiuzkr51c4n27/Tanks.csv?rlkey=tlgb1dnlgyv4yae7pqnne4tsh&raw=1')

In [ ]:
# Display dataframes to visualize what the data includes
display(df_fuel_level_1)
display(df_fuel_level_2)
display(df_invoices)
display(df_locations)
display(df_tanks)

## Describing the Data: Cleaning & Merging

In [ ]:
# Data Cleaning Part 1
# Let's first start with ensuring the df_fuel_level_1 is cleaned

df_fuel_level_1.head() # Displays the first few rows of df_fuel_1 to examine its structure

In [ ]:
# Check for and count duplicated rows in the df_fuel_level_1 DataFrame, returning the sum
count_1 = df_fuel_level_1.duplicated().sum()
print(count_1)

# Remove duplicate rows in df_fuel_level_1, making changes in-place.
df_fuel_level_1.drop_duplicates(inplace=True)

# Check for missing values (NaN) in each column of df_fuel_level_1 and return the sum
missing_count_1 = df_fuel_level_1.isna().sum()
print(missing_count_1)

# Remove rows containing missing values (NaN) in df_fuel_level_1, making changes in-place
df_fuel_level_1.dropna(inplace=True)

# After removing duplicate rows and rows with missing values, we now want to reset the index of the DataFrame
df_fuel_level_1.reset_index(drop=True)

In [ ]:
# Data Cleaning Part 2 
# Let's first start with ensuring the df_fuel_level_2 is cleaned

df_fuel_level_2.head() # Displays the first few rows of df_fuel_2 to examine its structure

In [ ]:
# Check for and count duplicated rows in the df_fuel_level_2 DataFrame, returning a sum
count_2 = df_fuel_level_2.duplicated().sum()
print(count_2)

# Remove duplicate rows in df_fuel_level_2, making changes in-place
df_fuel_level_2.drop_duplicates(inplace=True)

# Check for missing values (NaN) in each column of df_fuel_level_2 and return the sum
missing_count_2 = df_fuel_level_2.isna().sum()
print(missing_count_2)

In [ ]:
# Data Cleaning Part 3
# Let's merge df_fuel_level_1 and df_fuel_level_2

# Rename columns in df_fuel_level_1 to match the column names in df_fuel_level_2
df_fuel_level_1.rename({'Tank ID': 'Tank_ID',
                        'Fuel Level': 'Fuel_Level',
                        'Time stamp': 'Time_stamp'}, axis=1, inplace=True)

# Rename columns in df_fuel_level_2 to match the column names in df_fuel_level_1
df_fuel_level_2.rename({'Tank ID': 'Tank_ID',
                        'Timestamp': 'Time_stamp'}, axis=1, inplace=True)

# Merge the two dataframes (df_fuel_level_1 and df_fuel_level_2) into a single dataframe, df_total
df_total = pd.concat([df_fuel_level_1, df_fuel_level_2])

In [ ]:
# Display information about the DataFrame df_total
df_total.info()

In [ ]:
# Convert the 'Time_stamp' column to datetime format
df_total.Time_stamp = pd.to_datetime(df_total.Time_stamp)

In [ ]:
# Data Cleaning Part 4
# Let's focus on df_invoices

# Display information about the DataFrame df_invoices
display(df_invoices)

In [ ]:
# Remove duplicate rows from the df_invoices DataFrame
df_invoices.drop_duplicates(inplace=True)

# Check for and count duplicated 'Invoice ID' values in the df_invoices DataFrame
count_3 = df_invoices['Invoice ID'].duplicated().sum()
print(count_3)

# Check for missing values (NaN) in each column of df_invoices and return sum
missing_count_3 = df_invoices.isna().sum()
print(missing_count_3)

In [ ]:
# Visualize missing data in the DataFrame df_invoices using the msno.matrix function
msno.matrix(df_invoices);

In [ ]:
# Identify and examine rows in the DataFrame where at least one column has missing values
missing_rows = df_invoices[df_invoices.isna().any(axis=1)]
print(missing_rows)

# Drop rows with missing values (NaN) from df_invoices
df_invoices.dropna(how='any', 
                   inplace=True)

In [ ]:
# Create a new column 'Cost_per_liter' in the df_invoices DataFrame (this gives us better insights)
# We divide the 'Gross Purchase Cost' by the 'Amount Purchased' for each row
df_invoices['Cost_per_liter'] = df_invoices['Gross Purchase Cost'].div(df_invoices['Amount Purchased'])

In [ ]:
# Let's look at the remaining DataFrames

# Display the contents of the df_locations DataFrame
display(df_locations)

# Display the contents of the df_tanks DataFrame
display(df_tanks)

In [ ]:
# Data Cleaning Part 5
# Merging all relevant data sets

# Merge df_total with df_tanks based on the 'Tank_ID' and 'Tank ID' columns
df_total = df_total.merge(df_tanks,
               left_on='Tank_ID',
               right_on='Tank ID',
               how='left',
               validate='m:1')

# Rename columns to match a consistent naming convention across datasets
df_total.rename({'Tank Location': 'Tank_Location',
                'Tank Number': 'Tank_Number',
                'Tank Type': 'Tank_Type',
                'Tank Capacity': 'Tank_Capacity'}, axis=1, inplace=True)

# Drop the 'Tank ID' column as it's no longer needed after the merge
df_total.drop('Tank ID', axis=1, inplace=True)

# Merge df_total with df_locations to add location information
df_total = df_total.merge(df_locations[['Gas Station Location', 'Gas Station Name']],
                          left_on='Tank_Location',
                          right_on='Gas Station Location',
                          how='left',
                          validate='m:1')

# Drop the 'Gas Station Location' column as it's no longer needed
df_total.drop('Gas Station Location', axis=1, inplace=True)

# Rename the 'Gas Station Name' column to 'Station_Name' for consistency
df_total.rename({'Gas Station Name': 'Station_Name'}, axis=1, inplace=True)

In [ ]:
# Convert the 'Invoice Date' column to a datetime data type using pd.to_datetime
df_invoices['Invoice Date'] = pd.to_datetime(df_invoices['Invoice Date'])

# Create a new column 'Day_of_Week' to extract and store the day of the week from the 'Invoice Date'
df_invoices['Day_of_Week'] = df_invoices['Invoice Date'].dt.day_name()

## Splitting the Data into Manageable Parts

In [ ]:
# Splitting Data by Tank

# Create separate DataFrames for each tank (T10, T11, T12, ..., T32) based on 'Tank_ID'
df_T10 = df_total.groupby(by=['Tank_ID']).get_group('T 10').reset_index(drop=True)
df_T11 = df_total.groupby(by=['Tank_ID']).get_group('T 11').reset_index(drop=True)
df_T12 = df_total.groupby(by=['Tank_ID']).get_group('T 12').reset_index(drop=True)
df_T13 = df_total.groupby(by=['Tank_ID']).get_group('T 13').reset_index(drop=True)
df_T14 = df_total.groupby(by=['Tank_ID']).get_group('T 14').reset_index(drop=True)
df_T15 = df_total.groupby(by=['Tank_ID']).get_group('T 15').reset_index(drop=True)
df_T16 = df_total.groupby(by=['Tank_ID']).get_group('T 16').reset_index(drop=True)
df_T17 = df_total.groupby(by=['Tank_ID']).get_group('T 17').reset_index(drop=True)
df_T18 = df_total.groupby(by=['Tank_ID']).get_group('T 18').reset_index(drop=True)
df_T19 = df_total.groupby(by=['Tank_ID']).get_group('T 19').reset_index(drop=True)
df_T20 = df_total.groupby(by=['Tank_ID']).get_group('T 20').reset_index(drop=True)
df_T21 = df_total.groupby(by=['Tank_ID']).get_group('T 21').reset_index(drop=True)
df_T22 = df_total.groupby(by=['Tank_ID']).get_group('T 22').reset_index(drop=True)
df_T23 = df_total.groupby(by=['Tank_ID']).get_group('T 23').reset_index(drop=True)
df_T24 = df_total.groupby(by=['Tank_ID']).get_group('T 24').reset_index(drop=True)
df_T25 = df_total.groupby(by=['Tank_ID']).get_group('T 25').reset_index(drop=True)
df_T26 = df_total.groupby(by=['Tank_ID']).get_group('T 26').reset_index(drop=True)
df_T27 = df_total.groupby(by=['Tank_ID']).get_group('T 27').reset_index(drop=True)
df_T28 = df_total.groupby(by=['Tank_ID']).get_group('T 28').reset_index(drop=True)
df_T29 = df_total.groupby(by=['Tank_ID']).get_group('T 29').reset_index(drop=True)
df_T30 = df_total.groupby(by=['Tank_ID']).get_group('T 30').reset_index(drop=True)
df_T31 = df_total.groupby(by=['Tank_ID']).get_group('T 31').reset_index(drop=True)
df_T32 = df_total.groupby(by=['Tank_ID']).get_group('T 32').reset_index(drop=True)

In [ ]:
# Splitting Data by Fuel Location

# Create separate DataFrames for each fuel location (FL_1, FL_2, FL_3, ..., FL_8) based on 'Tank_Location'
df_FL_1 = df_total.groupby(by=['Tank_Location']).get_group(1).reset_index(drop=True)
df_FL_2 = df_total.groupby(by=['Tank_Location']).get_group(2).reset_index(drop=True)
df_FL_3 = df_total.groupby(by=['Tank_Location']).get_group(3).reset_index(drop=True)
df_FL_4 = df_total.groupby(by=['Tank_Location']).get_group(4).reset_index(drop=True)
df_FL_5 = df_total.groupby(by=['Tank_Location']).get_group(5).reset_index(drop=True)
df_FL_6 = df_total.groupby(by=['Tank_Location']).get_group(6).reset_index(drop=True)
df_FL_7 = df_total.groupby(by=['Tank_Location']).get_group(7).reset_index(drop=True)
df_FL_8 = df_total.groupby(by=['Tank_Location']).get_group(8).reset_index(drop=True)

In [ ]:
# Creating a DataFrame for Inflation Data

# Define a dictionary containing inflation rate data for specific months and years
inflation_rate = {
    'Date': ['January 2017', 'February 2017', 'March 2017', 'April 2017', 'May 2017',
             'June 2017', 'July 2017', 'August 2017', 'September 2017', 'October 2017',
             'November 2017', 'December 2017', 'January 2018', 'February 2018', 'March 2018',
             'April 2018', 'May 2018', 'June 2018', 'July 2018', 'August 2018',
             'September 2018', 'October 2018', 'November 2018', 'December 2018',
             'January 2019', 'February 2019', 'March 2019', 'April 2019', 'May 2019',
             'June 2019', 'July 2019', 'August 2019'],
    'Rate': [2.13, 2.05, 1.56, 1.64, 1.32, 1.01, 1.16, 1.4, 1.55, 1.39, 2.1, 1.87, 1.7, 2.16,
             2.31, 2.22, 2.22, 2.45, 2.99, 2.84, 2.22, 2.44, 1.68, 1.99, 1.44, 1.51, 1.88, 2.03,
             2.4, 2.02, 2.01, 1.94]
}

# Create a DataFrame, df_inflation, using the inflation_rate dictionary
df_inflation = pd.DataFrame(inflation_rate)

# Calculate an 'Inflation_Multiplier' column by dividing the 'Rate' by 100 and adding 1
df_inflation['Inflation_Multiplier'] = (df_inflation.Rate / 100) + 1

In [ ]:
# Merging Infaltion with Current Data

# Convert the 'Date' column in the df_inflation DataFrame to a datetime data type using pd.to_datetime
df_inflation['Date'] = pd.to_datetime(df_inflation['Date'])

# Create new columns 'Year' and 'Month' in df_inflation
df_inflation['Year'] = df_inflation['Date'].dt.year
df_inflation['Month'] = df_inflation['Date'].dt.month_name()

# Create 'Year' and 'Month' columns in df_invoices
df_invoices['Year'] = df_invoices['Invoice Date'].dt.year
df_invoices['Month'] = df_invoices['Invoice Date'].dt.month_name()

# Merge the df_invoices DataFrame with the df_inflation DataFrame
df_invoices = df_invoices.merge(df_inflation[['Month', 'Year', 'Inflation_Multiplier']],
                           on=['Month', 'Year'],
                           how='left',
                           validate='m:1')

In [ ]:
# Calculate 'Adjusted_cost' by dividing 'Cost_per_liter' by 'Inflation_Multiplier'
df_invoices['Adjusted_cost'] = df_invoices.Cost_per_liter.div(df_invoices.Inflation_Multiplier)

# Create separate DataFrames for each invoice location (Inv_1, Inv_2, Inv_3, ..., Inv_8) based on 'Invoice Gas Station Location'
df_Inv_1 = df_invoices[df_invoices['Invoice Gas Station Location'] == 1]
df_Inv_2 = df_invoices[df_invoices['Invoice Gas Station Location'] == 2]
df_Inv_3 = df_invoices[df_invoices['Invoice Gas Station Location'] == 3]
df_Inv_4 = df_invoices[df_invoices['Invoice Gas Station Location'] == 4]
df_Inv_5 = df_invoices[df_invoices['Invoice Gas Station Location'] == 5]
df_Inv_6 = df_invoices[df_invoices['Invoice Gas Station Location'] == 6]
df_Inv_7 = df_invoices[df_invoices['Invoice Gas Station Location'] == 7]
df_Inv_8 = df_invoices[df_invoices['Invoice Gas Station Location'] == 8]

In [ ]:
# Creating Temporary DataFrames for Invoice Analysis

# For each invoice location (Inv_1, Inv_2, ..., Inv_8), create temporary DataFrames (temp1, temp2, ..., temp8) 
temp1 = df_Inv_1.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp2 = df_Inv_2.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp3 = df_Inv_3.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp4 = df_Inv_4.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp5 = df_Inv_5.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp6 = df_Inv_6.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp7 = df_Inv_7.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()
temp8 = df_Inv_8.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()

In [ ]:
# Location vs Number of Invoices

temp = df_invoices.groupby(by=['Invoice Gas Station Location', 'Day_of_Week', 'Fuel Type'])['Invoice ID'].count().reset_index()

sns.set_style("whitegrid")  # Set the plot style to a white grid background
plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax = sns.barplot(data=temp, 
                 x='Invoice Gas Station Location', 
                 y='Invoice ID', 
                 hue='Fuel Type', 
                 palette = 'colorblind', 
                 errorbar=None)

# Add axis titles to the plot
plt.xlabel("Location (Gas Station)")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Number of Invoices by Gas Station Location", fontsize=16)

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Save the plot as a JPG image
plt.savefig("Invoices_By_Gas_Station.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Tank Number vs Tank Capacity

sns.set_style("whitegrid")  # Set the plot style to a white grid background
plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the horizontal bar plot
bar_plot = sns.barplot(data=df_tanks, 
                       x='Tank Capacity', 
                       y='Tank ID', 
                       palette='colorblind', 
                       hue= 'Tank Location', 
                       dodge = False)

# Add axis titles to the plot
plt.xlabel("Tank Capacity (in Liters)")
plt.ylabel("Tank ID")

# Add a title to the plot
plt.title("Tank Capacity by Tank ID", fontsize=16)

# Set the legend outside the plot for better readability
plt.legend(title="Tank Location", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Save the plot as a JPG image
plt.savefig("Tank_Capacity.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 1 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot for the first location (temp1).
ax1 = sns.barplot(data=temp1, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 1 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_1_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 2 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax1 = sns.barplot(data=temp2, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 2 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_2_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 3 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax1 = sns.barplot(data=temp3, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 3 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_3_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 4 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot 
ax1 = sns.barplot(data=temp4, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 4 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_4_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 5 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax1 = sns.barplot(data=temp5, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 5 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_5_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 6 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax1 = sns.barplot(data=temp6, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 6 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_6_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 7 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax1 = sns.barplot(data=temp7, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 7 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_7_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Days vs Location Invoices (Graph 8 of 8)

sns.set_style("whitegrid")  # Set the plot style to a white grid background

plt.figure(figsize=(12, 6))  # Set the figure size to control the plot's dimensions

# Create the bar plot
ax1 = sns.barplot(data=temp8, 
                  x='Day_of_Week', 
                  y='Invoice ID', 
                  hue='Fuel Type', 
                  errorbar=None, 
                  palette='colorblind')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Number of Invoices")

# Set the legend outside the plot for better readability
plt.legend(title="Fuel Type", title_fontsize='15', loc='upper left', bbox_to_anchor=(1, 1))

# Add a title to the plot
plt.title("Location 8 - Invoices by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("location_8_invoices.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

## In Depth Data Analysis: What is Going On?

In [ ]:
# What Day is the Cheapest?

sns.set_style("whitegrid") 
plt.figure(figsize=(10, 6))

# Group the data in the 'df_invoices' DataFrame by 'Day_of_Week' and calculate the mean of 'Adjusted_cost' for each day
bar_plot = df_invoices.groupby(by=['Day_of_Week']).Adjusted_cost.mean().plot(kind='bar', color='goldenrod')

# Rotate the x-axis labels for better readability
bar_plot.set_xticklabels(bar_plot.get_xticklabels(), rotation=45, ha="right")

# Add axis titles to the plot
plt.xlabel("Day of the Week")
plt.ylabel("Average Adjusted Cost")

# Add a title to the plot
plt.title("Average Adjusted Cost by Day of the Week", fontsize=16)

# Save the plot as a JPG image
plt.savefig("Cheapest_Days.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Diesel and Gas Breakdown (Side by Side)

sns.set_style("whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot the graph for Gas (G) on the left side
df_G = df_invoices.groupby(by=['Fuel Type']).get_group('G')
bar_plot_g = df_G.groupby(by=['Day_of_Week']).Adjusted_cost.mean().plot(kind='bar', color='forestgreen', ax=axes[0])

# Set axis titles for the Gas graph
axes[0].set_xlabel("Day of the Week")
axes[0].set_ylabel("Average Adjusted Cost")
axes[0].set_title("Gas (G) - Average Adjusted Cost by Day of the Week")

# Rotate the x-axis labels for the Gas graph
bar_plot_g.set_xticklabels(bar_plot_g.get_xticklabels(), rotation=45, ha="right")

# Plot the graph for Diesel (D) on the right side
df_D = df_invoices.groupby(by=['Fuel Type']).get_group('D')
bar_plot_d = df_D.groupby(by=['Day_of_Week']).Adjusted_cost.mean().plot(kind='bar', color='maroon', ax=axes[1])

# Set axis titles for the Diesel graph
axes[1].set_xlabel("Day of the Week")
axes[1].set_ylabel("Average Adjusted Cost")
axes[1].set_title("Diesel (D) - Average Adjusted Cost by Day of the Week")

# Rotate the x-axis labels for the Diesel graph.
bar_plot_d.set_xticklabels(bar_plot_d.get_xticklabels(), rotation=45, ha="right")

# Adjust the spacing between the two subplots.
plt.tight_layout()

# Save the combined graph as a JPG image
plt.savefig("diesel_gas_breakdown.jpg", dpi=250, bbox_inches="tight")

# Show the combined graph
plt.show()

In [ ]:
# What are the values of adjusted cost by day of the week?

# Group the data in the 'df_invoices' DataFrame by 'Day_of_Week' and calculate the mean of 'Adjusted_cost'
df_cost_per_day = df_invoices.groupby(by=['Day_of_Week']).Adjusted_cost.mean().reset_index()
df_cost_per_day = df_cost_per_day.sort_values(by='Adjusted_cost')
display(df_cost_per_day)

In [ ]:
# Assuming the Tanks can fill up to maximum capacity
# Are they really benefitting from the maximum discount they can get from the supplier?

print('Gas Station Location 1:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 1
print(f"For T10: {df_T10[df_T10.Fuel_Level <= 15000].shape[0] / df_T10.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T11: {df_T11[df_T11.Fuel_Level <= 15000].shape[0] / df_T11.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T12: {df_T12[df_T12.Fuel_Level <= 15000].shape[0] / df_T12.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T13: {df_T13[df_T13.Fuel_Level <= 15000].shape[0] / df_T13.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T14: {df_T14[df_T14.Fuel_Level <= 15000].shape[0] / df_T14.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T15: {df_T15[df_T15.Fuel_Level <= 15000].shape[0] / df_T15.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print()

print('Gas Station Location 2:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 2
print(f"For T16: {df_T16[df_T16.Fuel_Level <= 30000].shape[0] / df_T16.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T17: {df_T17[df_T17.Fuel_Level <= 15000].shape[0] / df_T17.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T18: {df_T18[df_T18.Fuel_Level <= 15000].shape[0] / df_T18.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T19: {df_T19[df_T19.Fuel_Level <= 30000].shape[0] / df_T19.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print()

print('Gas Station Location 3:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 3
print(f"For T20: {df_T20[df_T20.Fuel_Level <= 5000].shape[0] / df_T20.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T21: {df_T21[df_T21.Fuel_Level <= 5000].shape[0] / df_T21.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print()

print('Gas Station Location 4:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 4
print(f"For T22: {df_T22[df_T22.Fuel_Level <= 15000].shape[0] / df_T22.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T23: {df_T23[df_T23.Fuel_Level <= 15000].shape[0] / df_T23.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print()

print('Gas Station Location 5:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 5
print(f"For T24: {df_T24[df_T24.Fuel_Level <= 10000].shape[0] / df_T24.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T25: {df_T25[df_T25.Fuel_Level <= 10000].shape[0] / df_T25.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print()

print('Gas Station Location 6:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 6
print(f"For T26: {df_T26[df_T26.Fuel_Level <= 5000].shape[0] / df_T26.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T27: {df_T27[df_T27.Fuel_Level <= 5000].shape[0] / df_T27.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T28: {df_T28[df_T28.Fuel_Level <= 5000].shape[0] / df_T28.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print()

print('Gas Station Location 8:')
# Calculate the percentage of replenishments benefiting from the maximum discount for each tank at Gas Station Location 8
print(f"For T31: {df_T31[df_T31.Fuel_Level <= 15000].shape[0] / df_T31.shape[0]:.2%} of the replenishments benefited from the maximum discount")
print(f"For T32: {df_T32[df_T32.Fuel_Level <= 15000].shape[0] / df_T32.shape[0]:.2%} of the replenishments benefited from the maximum discount")

### Note: Gas Station Location 7 has two tanks of capacity 5000 each. Thus, they cannot benefit from any supplier discounts.

## Vizualization of Tank Capacities: What does usage and replenishment look like?
- We will use the examples of T20 (cap = 30,000), T12 (cap = 40,000), and T16 (cap = 70,000)

In [ ]:
# For Tank 20 of capacity 30,000

# Create a new figure and axis for the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the Fuel Level over time for Tank 20
ax.plot(df_T20.Time_stamp, df_T20.Fuel_Level, label='Fuel Level', color='slateblue')

# Add a horizontal line at 5000 for ideal level at which the gas station should refill
ax.axhline(5000, color='brown', linestyle='--', label='Minimum Fuel Level')

# Set plot title
ax.set_title("Fuel Level Over Time for Tank 20", fontsize=16)

# Set axis labels
ax.set_xlabel("Time Stamp")
ax.set_ylabel("Fuel Level")

# Add legend to the plot
ax.legend()

# Save the plot as a JPG image
plt.savefig("Tank20_Fuel_Level.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# For Tank 12 of capacity 40,000

# Create a new figure and axis for the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the Fuel Level over time for Tank 12
ax.plot(df_T12.Time_stamp, df_T12.Fuel_Level, label='Fuel Level', color='slateblue')

# Add a horizontal line at 15000 for ideal level at which the gas station should refill
ax.axhline(15000, color='brown', linestyle='--', label='Minimum Fuel Level')

# Set plot title
ax.set_title("Fuel Level Over Time for Tank 12", fontsize=16)

# Set axis labels
ax.set_xlabel("Time Stamp")
ax.set_ylabel("Fuel Level")

# Add legend to the plot
ax.legend()

# Save the plot as a JPG image
plt.savefig("Tank12_Fuel_Level.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# For Tank 16 of capacity 70,000

# Create a new figure and axis for the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the Fuel Level over time for Tank 16
ax.plot(df_T16.Time_stamp, df_T16.Fuel_Level, label='Fuel Level', color='slateblue')

# Add a horizontal line at 30000 for ideal level at which the gas station should refill
ax.axhline(30000, color='brown', linestyle='--', label='Minimum Fuel Level')

# Set plot title
ax.set_title("Fuel Level Over Time for Tank 16", fontsize=16)

# Set axis labels
ax.set_xlabel("Time Stamp")
ax.set_ylabel("Fuel Level")

# Add legend to the plot
ax.legend()

# Save the plot as a JPG image
plt.savefig("Tank16_Fuel_Level.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Are locations being efficient when ordering?
# Is there ever an instance where they have to make two orders in one day?

# Group the invoices by 'Invoice Date' and count the number of occurrences for each combination of location and fuel type
df_temp = df_invoices.groupby(by=['Invoice Date'])[['Invoice Gas Station Location', 'Fuel Type']].value_counts().reset_index()

# Filter for instances where there are at least two orders on the same day for the same location and fuel type
df_temp = df_temp[df_temp[0] >= 2]

# Count the occurrences of multiple orders in one day for each location
multiple = df_temp['Invoice Gas Station Location'].value_counts().reset_index()

# Display the results
display(multiple)

### There are 5 gas station locations that ordered the same fuel type more than once per day.

In [ ]:
# Vizualize the amount of gas stations ordering more than once per day
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=multiple, x='index', y='Invoice Gas Station Location', palette='colorblind')

# Add axis titles
plt.xlabel("Gas Station Location")
plt.ylabel("Number of Days with Multiple Orders")
plt.title("Efficiency of Gas Station Locations in Ordering", fontsize=16)

# Save the plot as a JPG image
plt.savefig("Ordering_Efficiency.jpg", dpi=250, bbox_inches="tight")

# Show the plot
plt.show()

### Which location orders (more than once a day) the most? Location 1

## Price Calculations: What are the potential savings?

### We will be focusing on Gas Station 1 that has the following tanks: 
- T10 
- T11
- T12 
- T13
- T14 
- T15

### Note: This same methedology can be replicated for locations 2 - 8, gathering even more potential savings

In [ ]:
# Goal: To find the average Consumption per day and per week

# Step 1:
# Extract the date component from the 'Time_stamp' column for each tank location
df_T10['Date'] = df_T10['Time_stamp'].dt.date
df_T11['Date'] = df_T11['Time_stamp'].dt.date
df_T12['Date'] = df_T12['Time_stamp'].dt.date
df_T13['Date'] = df_T13['Time_stamp'].dt.date
df_T14['Date'] = df_T14['Time_stamp'].dt.date
df_T15['Date'] = df_T15['Time_stamp'].dt.date

In [ ]:
# Step 2:
# Calculate the differences in fuel levels for all tanks

df_T10['Fuel_Level_Difference'] = df_T10['Fuel_Level'].diff()
df_T11['Fuel_Level_Difference'] = df_T11['Fuel_Level'].diff()
df_T12['Fuel_Level_Difference'] = df_T12['Fuel_Level'].diff()
df_T13['Fuel_Level_Difference'] = df_T13['Fuel_Level'].diff()
df_T14['Fuel_Level_Difference'] = df_T14['Fuel_Level'].diff()
df_T15['Fuel_Level_Difference'] = df_T15['Fuel_Level'].diff()

In [ ]:
# Step 3:
# Calculate and print the average daily fuel consumption for all tanks

print(f"The average consumption per day for T10 is \
{round((df_T10[df_T10['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T10['Date'].nunique(), 2)} \
liters per day")
print()

print(f"The average consumption per day for T11 is \
{round((df_T11[df_T11['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T11['Date'].nunique(), 2)} \
liters per day")
print()

print(f"The average consumption per day for T12 is \
{round((df_T12[df_T12['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T12['Date'].nunique(), 2)} \
liters per day")
print()

print(f"The average consumption per day for T13 is \
{round((df_T13[df_T13['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T13['Date'].nunique(), 2)} \
liters per day")
print()

print(f"The average consumption per day for T14 is \
{round((df_T14[df_T14['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T14['Date'].nunique(), 2)} \
liters per day")
print()

print(f"The average consumption per day for T15 is \
{round((df_T15[df_T15['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T15['Date'].nunique(), 2)} \
liters per day")
print()

In [ ]:
# Step 4/ Final Step:
# Calculate and print the average weekly fuel consumption for all tanks

print(f"The average weekly comsumption for T10 is \
{round(((df_T10[df_T10['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T10['Date'].nunique()) * 7,2)}\
liters per day")
print()

print(f"The average weekly comsumption for T11 is \
{round(((df_T11[df_T11['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T11['Date'].nunique())*7, 2)}\
liters per day")
print()

print(f"The average weekly comsumption for T12 is \
{round(((df_T12[df_T12['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T12['Date'].nunique())*7, 2)}\
liters per day")
print()

print(f"The average weekly comsumption for T13 is \
{round(((df_T13[df_T13['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T13['Date'].nunique())*7, 2)}\
liters per day")
print()

print(f"The average weekly comsumption for T14 is \
{round(((df_T14[df_T14['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T14['Date'].nunique())*7, 2)}\
liters per day")
print()

print(f"The average weekly comsumption for T15 is \
{round(((df_T15[df_T15['Fuel_Level_Difference'] < 0]['Fuel_Level_Difference'].mul(-1).sum()) / df_T15['Date'].nunique())*7, 2)}\
liters per day")
print()

### Based on previous graphs, we know that all of the tanks at Location 1 have a capacity of 40,000. However since the monthly consumption of all is below 40,000, then they can order once a week (between 25000 and 40000) to benefit from the maximum discount (which in this case in 3 cents per liter)

### The Recommendation:
- For Tank T10, they could order on average 25000 liters once a week on Saturday (so 4 times a month)
- For Tank T11, they could order on average 35000 liters once a week on Saturday (so 4 times a month)
- For Tank 12, they could order on average 35000 liters once a week on Saturday (so 4 times a month)
- For Tank 13, they could order on average 25000 liters once every two weeks on Saturday (so 2 times a month)
- For Tank 14, they could order on average 25000 liters once a week on Saturday (so 4 times a month)
- For Tank 15, they could order on average 35000 liters once every two weeks on Saturday (so 2 times a month)

In [ ]:
# Calculate the total  of number replenishments based on the number of unique tanks

total_replenishments = (
    4 * (df_T10.Date.nunique()/30) + 
    4 * (df_T11.Date.nunique()/30) + 
    4 * (df_T12.Date.nunique()/30) + 
    2 * (df_T13.Date.nunique()/30) +
    4 * (df_T14.Date.nunique()/30) +
    2 * (df_T15.Date.nunique()/30)
)

# Round the total replenishments to the nearest whole number
total_replenishments = round(total_replenishments)

print(f"Thus, following our recommendation we can conclude that Location 1 could have had {total_replenishments} replenishments \n(over the last 2 and a half years) instead of {df_Inv_1.shape[0]} replenishments")

In [ ]:
# How much would it cost (per gallon) if Location 1 followed our recommendations?
# Note: This means that they would get 3 cents off per liter

# Access the value at the first row (index 0) and the second column (index 1) df_cost_per_day
initial_price = df_cost_per_day.iloc[0][1]

# Subtract 0.03 from the initial price to apply a 3 cent discount
discounted_price = initial_price - 0.03

# Round the discounted_price to three decimal places
discounted_price = discounted_price.round(3)

print(discounted_price)

In [ ]:
# The average cost of replenishment per tank, per month

# Calculate the monthly cost for tank T10 by multiplying the refill amount (25000) by 4 (to represent 4 weeks)
# and then by the discounted price
monthly_cost_T10 = 25000 * 4 * discounted_price

# Calculate the monthly cost for tank T11 by multiplying the refill amount (35000) by 4 (to represent 4 weeks)
# and then by the discounted price
monthly_cost_T11 = 35000 * 4 * discounted_price

# Calculate the monthly cost for tank T12 by multiplying the refill amount (35000) by 4 (to represent 4 weeks)
# and then by the discounted price
monthly_cost_T12 = 35000 * 4 * discounted_price

# Calculate the monthly cost for tank T13 by multiplying the refill amount (25000) by 2 (to represent twice a month)
# and then by the discounted price
monthly_cost_T13 = 25000 * 2 * discounted_price

# Calculate the monthly cost for tank T14 by multiplying the refill amount (25000) by 4 (to represent 4 weeks)
# and then by the discounted price
monthly_cost_T14 = 25000 * 4 * discounted_price

# Calculate the monthly cost for tank T15 by multiplying the refill amount (35000) by 4 (to represent twice a month)
# and then by the discounted price
monthly_cost_T15 = 35000 * 2 * discounted_price

print(f"The average monthly estimated cost of replenishement for Tank T10 is ${monthly_cost_T10:,.2f}")
print(f"The average monthly estimated cost of replenishement for Tank T11 is ${monthly_cost_T11:,.2f}")
print(f"The average monthly estimated cost of replenishement for Tank T12 is ${monthly_cost_T12:,.2f}")
print(f"The average monthly estimated cost of replenishement for Tank T13 is ${monthly_cost_T13:,.2f}")
print(f"The average monthly estimated cost of replenishement for Tank T14 is ${monthly_cost_T14:,.2f}")
print(f"The average monthly estimated cost of replenishement for Tank T15 is ${monthly_cost_T15:,.2f}")

In [ ]:
# Thus, now that we have the monthly average, let's multiply the data by 2 1/2 years to get the total cost per tank

estimated_cost_T10 = monthly_cost_T10 * (df_T10.Date.nunique()/30)
estimated_cost_T11 = monthly_cost_T11 * (df_T11.Date.nunique()/30)
estimated_cost_T12 = monthly_cost_T12 * (df_T12.Date.nunique()/30)
estimated_cost_T13 = monthly_cost_T13 * (df_T13.Date.nunique()/30)
estimated_cost_T14 = monthly_cost_T14 * (df_T14.Date.nunique()/30)
estimated_cost_T15 = monthly_cost_T15 * (df_T15.Date.nunique()/30)

print(f"The total estimated cost of replenishment for tank T10 for the 2 1/2 years is ${estimated_cost_T10:,.2f}")
print(f"The total estimated cost of replenishment for tank T11 for the 2 1/2 years is ${estimated_cost_T11:,.2f}")
print(f"The total estimated cost of replenishment for tank T12 for the 2 1/2 years is ${estimated_cost_T12:,.2f}")
print(f"The total estimated cost of replenishment for tank T13 for the 2 1/2 years is ${estimated_cost_T13:,.2f}")
print(f"The total estimated cost of replenishment for tank T14 for the 2 1/2 years is ${estimated_cost_T14:,.2f}")
print(f"The total estimated cost of replenishment for tank T15 for the 2 1/2 years is ${estimated_cost_T15:,.2f}")

In [ ]:
# Now, let's add up all the values to get the total cost of all tanks in location 1
estimated_total_1 = estimated_cost_T10 + estimated_cost_T11 + estimated_cost_T12+ estimated_cost_T13+ estimated_cost_T14+ estimated_cost_T15
print(estimated_total_1)

# For comparison, we also need the total amount spent (at location 1) to date
total_location_1 = df_Inv_1['Gross Purchase Cost'].sum()
print(total_location_1)

## Thus, Location 1 cound have paid `$18,042,310` instead of `$18,198,952.88`

## Giving us `$156,642.88` in savings from just one location!